# Doc2vecの文書ベクトルを使用し、scikit-learnでコサイン類似検索




- Wikipediaの文書からDoc2Vecモデルを生成


- scikit-learnのcosine-simularityを使用して類似検索


下記レポート作成時のモデルファイルをそのまま使用しています。

<a href="23-Doc2Vec-with-Wiki.ipynb"><b>23-Doc2Vec-with-Wiki.ipynb</b></a>


nosetestsの質問文を使用してテストしたところ、４問中１問だけが正解、という結果に終わりました。

## (1) Wikipediaコンテンツファイルから全文書を抽出

レポート <a href="31-Wikipedia-contents-csv.ipynb"><b>31-Wikipedia-contents-csv.ipynb</b></a> の手順にて、いったんローカルPCにCSVファイル化しておきます。

## (2) Wikipedia文書を学習

レポート <a href="23-Doc2Vec-with-Wiki.ipynb"><b>23-Doc2Vec-with-Wiki.ipynb</b></a> の手順にて生成したDoc2Vecモデルファイルをロードして使用します。

上記手順では、Wikipedia文書のみを使用し、ボキャブラリ／単語ベクトルの生成および学習を行い、モデルをファイル保存しています。

In [1]:
'''
    環境準備
'''
import sys
import os

import numpy as np
import pandas as pd
 
learning_dir = os.path.abspath("../../") #<--- donusagi-bot/learning
os.chdir(learning_dir)
if learning_dir not in sys.path:
    sys.path.append(learning_dir)

In [2]:
from gensim import models
from gensim.models.doc2vec import Doc2Vec

def doc2vec_model_path(dm):
    model_path = 'prototype/better_algorithm/doc2vec.wikipedia.PV%d.model' % dm
    return model_path

In [3]:
'''
    あらかじめ学習したモデルのファイルをロード
    dm = 0 : DBoWを使用したモデル
'''
dm = 0
loaded_model_dbow = models.Doc2Vec.load(doc2vec_model_path(dm))

print('Document vector size=%d' % (len(loaded_model_dbow.docvecs)))

Document vector size=87181


## (3) my-opeの文書を、Wikiから生成したモデルにより文書ベクトル化する関数

Wikipedia文書だけで学習されたDoc2Vecモデルを使用し、my-ope文書（質問文）をベクトル化します。

In [4]:
import numpy as np

from learning.core.learn.learning_parameter import LearningParameter
from learning.core.datasource import Datasource

_bot_id = 13 # toyotsu_human.csv
attr = {
    'include_failed_data': False,
    'include_tag_vector': False,
    'classify_threshold': 0.5,
    'algorithm': LearningParameter.ALGORITHM_LOGISTIC_REGRESSION,
    'params_for_algorithm': {'C': 140},
    'excluded_labels_for_fitting': None
}
learning_parameter = LearningParameter(attr)

In [5]:
from learning.core.nlang import Nlang

In [6]:
def get_document_vector(question, model, warning):
    '''
        question: 
            分かち書きされていない文書
        model:
            Doc2Vecの学習済みモデル
            （検証時は品詞を落としていないWikipedia文書からモデルを生成）

        inferred_vector:
            文書を分かち書きしたコーパスから、
            Doc2Vecの学習済みモデルを使用して
            生成される類似文書ベクトル
            （learning.core.nlang.Nlangの仕様に従い、
            　一部品詞が落とされます。）

        warning:
            Trueを指定時、コーパスに含まれる単語が
            モデル内のWord2Vecボキャブラリにない場合、
            警告を表示する
    '''
    corpus = Nlang.split(question).split()
    inferred_vector = model.infer_vector(corpus)
    
    if warning:
        for c in corpus:
            if not c in model.wv.vocab:
                print("Warning: word [%s] does not exist in Word2Vec vocabulary." % c)

    return inferred_vector

def get_document_vectors(questions, model, warning=False):
    document_vectors = []
    for question in questions:
        inferred_vector = get_document_vector(question, model, warning)
        document_vectors.append(list(inferred_vector))

    return np.array(document_vectors)

## (4) コサイン類似検索の実行

質問文は、my-ope プロダクションの nosetests テストケースから引用しました。

In [7]:
from sklearn.metrics.pairwise import cosine_similarity
from learning.core.datasource import Datasource

def search_simiarity(question, dbow_model):
    '''
        質問文間でコサイン類似度を算出して、近い質問文の候補を取得する
        
        仕様はプロダクションに準拠しています
        ただし、文書のベクトル化は、TF-IDFではなく、
        Doc2Vecを使用します。
    '''
    datasource = Datasource('csv')
    question_answers = datasource.question_answers_for_suggest(_bot_id, question)

    #all_array = TextArray(question_answers['question'], vectorizer=self.vectorizer)
    #question_array = TextArray([question], vectorizer=self.vectorizer)
    all_array      = get_document_vectors(question_answers['question'], dbow_model)
    question_array = get_document_vectors([question], dbow_model, warning=True)
    
    print('count: my-ope all questions=%d, document vectors=%d (features=%d)' % (
        np.size(question_answers['question']), all_array.shape[0], all_array.shape[1]
    ))    
    print('count: question=%d, document vectors=%d (features=%d)' % (
        np.size([question]), question_array.shape[0], question_array.shape[1]
    ))    

    similarities = cosine_similarity(all_array, question_array)
    similarities = similarities.flatten()

    ordered_result = list(map(lambda x: {
        'question_answer_id': float(x[0]), 'similarity': x[1], 'answer_id': x[2]
    }, sorted(zip(question_answers['id'], similarities, question_answers['answer_id']), key=lambda x: x[1], reverse=True)))

    df = pd.DataFrame.from_dict(ordered_result)

    print(df[0:10])

In [8]:
# 正解＝6803
search_simiarity('JAL マイレージ', loaded_model_dbow)

2017/05/30 PM 01:30:48 ['./fixtures/learning_training_messages/benefitone.csv', './fixtures/learning_training_messages/ptna.csv', './fixtures/learning_training_messages/septeni.csv', './fixtures/learning_training_messages/toyotsu_human.csv']
2017/05/30 PM 01:30:48 ['./fixtures/question_answers/toyotsu_human.csv']


count: my-ope all questions=317, document vectors=317 (features=200)
count: question=1, document vectors=1 (features=200)
   answer_id  question_answer_id  similarity
0       6976             13556.0    0.173428
1       6910             13484.0    0.162010
2       6949             13529.0    0.161308
3       6946             13525.0    0.160166
4       6738             13309.0    0.156441
5       6934             13513.0    0.152932
6       6892             13466.0    0.152278
7       6865             13439.0    0.145302
8       6779             13352.0    0.143245
9       6713             13283.0    0.143054


In [9]:
# 正解＝6763
search_simiarity('海外の出張費の精算の方法は？', loaded_model_dbow)

2017/05/30 PM 01:30:48 ['./fixtures/learning_training_messages/benefitone.csv', './fixtures/learning_training_messages/ptna.csv', './fixtures/learning_training_messages/septeni.csv', './fixtures/learning_training_messages/toyotsu_human.csv']
2017/05/30 PM 01:30:48 ['./fixtures/question_answers/toyotsu_human.csv']


count: my-ope all questions=317, document vectors=317 (features=200)
count: question=1, document vectors=1 (features=200)
   answer_id  question_answer_id  similarity
0       6856             13430.0    0.516781
1       6763             13335.0    0.505366
2       6888             13462.0    0.493470
3       6786             13359.0    0.463074
4       6744             13316.0    0.458465
5       6863             13437.0    0.455485
6       6762             13334.0    0.444064
7       6733             13304.0    0.439248
8       6824             13398.0    0.436261
9       6749             13310.0    0.433721


In [10]:
# 正解＝6767
search_simiarity('VISAの勘定科目がわからない', loaded_model_dbow) 

2017/05/30 PM 01:30:48 ['./fixtures/learning_training_messages/benefitone.csv', './fixtures/learning_training_messages/ptna.csv', './fixtures/learning_training_messages/septeni.csv', './fixtures/learning_training_messages/toyotsu_human.csv']
2017/05/30 PM 01:30:48 ['./fixtures/question_answers/toyotsu_human.csv']


count: my-ope all questions=317, document vectors=317 (features=200)
count: question=1, document vectors=1 (features=200)
   answer_id  question_answer_id  similarity
0       6787             13360.0    0.356194
1       6893             13467.0    0.354752
2       6799             13374.0    0.344399
3       6767             13339.0    0.338896
4       6772             13361.0    0.336147
5       6862             13436.0    0.324125
6       6796             13371.0    0.321354
7       6798             13373.0    0.320444
8       6900             13474.0    0.306015
9       6912             13489.0    0.305509


In [11]:
# 正解＝6909
search_simiarity('子供が生まれた', loaded_model_dbow) 

2017/05/30 PM 01:30:49 ['./fixtures/learning_training_messages/benefitone.csv', './fixtures/learning_training_messages/ptna.csv', './fixtures/learning_training_messages/septeni.csv', './fixtures/learning_training_messages/toyotsu_human.csv']
2017/05/30 PM 01:30:49 ['./fixtures/question_answers/toyotsu_human.csv']


count: my-ope all questions=317, document vectors=317 (features=200)
count: question=1, document vectors=1 (features=200)
   answer_id  question_answer_id  similarity
0       6909             13483.0    0.415835
1       6723             13294.0    0.410651
2       6956             13536.0    0.337485
3       6958             13538.0    0.315778
4       6980             13560.0    0.307432
5       6949             13529.0    0.304626
6       6988             13568.0    0.301286
7       7011             13591.0    0.300313
8       6957             13537.0    0.288305
9       6912             13488.0    0.283848
